In [1]:
from box import ConfigBox

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path

In [3]:
%cd TLDR

[Errno 2] No such file or directory: 'TLDR'
/home/dahir/deedax/TLDR/research


In [5]:
from TLDR.constants import *
from TLDR.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
  def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    
    create_directories([self.config.artifacts_root])

  def get_data_ingestion_config(self) -> DataIngestionConfig:
    config = self.config.data_ingestion
    create_directories([config.root_dir])

    data_ingestion_config = DataIngestionConfig(
      root_dir = config.root_dir,
      source_URL = config.source_URL,
      local_data_file = config.local_dir_file,
      unzip_dir = config.unzip_dir
    )

    return data_ingestion_config

In [16]:
import os
import urllib.request as request
import zipfile
from TLDR.logging import logger
from TLDR.utils.common import get_size

In [17]:
class DataIngestion:
  def __init__(self, config: DataIngestionConfig):
    self.config = config

  def download_file(self):
    if not os.path.exists(self.config.local_data_file):
      filename, headers = request.urlretrieve(
        url = self.config.source_URL,
        filename = self.config.local_data_file
      )
      logger.info(f"Downloaded {filename} with info \n{headers}")
    else:
      logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}")

  def extract_zip(self):
    unzip_dir = self.config.unzip_dir
    os.makedirs(unzip_dir, exist_ok=True)
    with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
      zip_ref.extractall(unzip_dir)

In [19]:
try:
  config_manager = ConfigutationManager()
  data_ingestion_config = config_manager.get_data_ingestion_config()
  data_ingestion = DataIngestion(config = data_ingestion_config)
  data_ingestion.download_file()
  data_ingestion.extract_zip()
except Exception as e:
  raise e

[2023-10-17 20:48:48,805: INFO: common] yaml file: <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> read successfully.]
[2023-10-17 20:48:48,808: INFO: common] yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> read successfully.]
[2023-10-17 20:48:48,810: INFO: common] Created directory: artifacts]
[2023-10-17 20:48:48,813: INFO: common] Created directory: artifacts/data_ingestion]
[2023-10-17 20:48:58,400: INFO: 1087526126] Downloaded artifacts/data_ingestion/data.zip with info 
Server: nginx/1.25.1
Date: Tue, 17 Oct 2023 19:48:55 GMT
Transfer-Encoding: chunked
Connection: close

]


In [13]:
os.chdir('..')